In [1]:
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent 
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
import subprocess 

In [2]:

help(convert_mcp_to_langchain_tools)


Help on function convert_mcp_to_langchain_tools in module langchain_mcp_tools.langchain_mcp_tools:

async convert_mcp_to_langchain_tools(server_configs: dict[str, langchain_mcp_tools.langchain_mcp_tools.McpServerCommandBasedConfig | langchain_mcp_tools.langchain_mcp_tools.McpServerUrlBasedConfig], logger: logging.Logger | int | None = None) -> tuple[list[langchain_core.tools.base.BaseTool], typing.Callable[[], typing.Awaitable[NoneType]]]
    Initialize multiple MCP servers and convert their tools to LangChain format.
    
    This is the main entry point for the library. It orchestrates the complete
    lifecycle of multiple MCP server connections, from initialization through
    tool conversion to cleanup. Provides robust error handling and authentication
    pre-validation to prevent common MCP client library issues.
    
    Key Features:
    - Parallel initialization of multiple servers for efficiency
    - Authentication pre-validation for HTTP servers to prevent async generator 

In [3]:
async def main():
    server_configs = {
        "add_tool": {
            "command": "D:\\Work\\miniconda_env\\try-mcp-2.x\\python.exe",
            "args": ["D:\\Work\\wsl\\Git\\LearnAgenticAI\\mcp\\example-3\\server\\add_server.py"],
            "transport": "stdio",
            "errlog": subprocess.DEVNULL 
        },
        "multiply_tool": {
            "url": "http://127.0.0.1:8000/mcp",
            "transport": "streamable-http",
            "errlog": subprocess.DEVNULL 
        }
    }

    tools, cleanup = await convert_mcp_to_langchain_tools(server_configs)


    print("Available tools:", tools)

    llm   = ChatOllama(model="llama3.2:3b", temperature=0)
    agent = create_react_agent(llm, tools)

    # Example queries
    for query in ["What is 7 + 13?", "Compute 6 × 9."]:
        result = await agent.ainvoke(
            {"messages": [{"role": "user", "content": query}]}
        )
        last   = result["messages"][-1]
        answer = last.get("content") if isinstance(last, dict) else last.content
        print(f"Query: {query}\nAnswer: {answer}\n")

    # Clean up MCP server sessions
    await cleanup()

In [4]:
await main()

[INFO] MCP server "add_tool": initializing with: {'command': 'D:\\Work\\miniconda_env\\try-mcp-2.x\\python.exe', 'args': ['D:\\Work\\wsl\\Git\\LearnAgenticAI\\mcp\\example-3\\server\\add_server.py'], 'transport': 'stdio', 'errlog': -3}
[INFO] MCP server "add_tool": spawning local process via stdio
[INFO] MCP server "multiply_tool": initializing with: {'url': 'http://127.0.0.1:8000/mcp', 'transport': 'streamable-http', 'errlog': -3}
[INFO] MCP server "multiply_tool": Pre-validating authentication
[INFO] MCP server "multiply_tool": Authentication validation passed: 307
[INFO] MCP server "multiply_tool": testing Streamable HTTP support for http://127.0.0.1:8000/mcp
[INFO] MCP server "multiply_tool": detected Streamable HTTP transport support
[INFO] MCP server "add_tool": connected
[INFO] MCP server "add_tool": 1 tool(s) available:
[INFO] - add
[INFO] Received session ID: be2c986267d448e188a152e20d0aaf4b
[INFO] Negotiated protocol version: 2025-06-18
[INFO] MCP server "multiply_tool": conn

Available tools: [McpToLangChainAdapter(), McpToLangChainAdapter()]


[INFO] MCP tool "add_tool"/"add" received input: {'a': 7, 'b': 13}
[INFO] MCP tool "add_tool"/"add" received result (size: 2)


Query: What is 7 + 13?
Answer: The answer to the original question, "What is 7 + 13?", is 20.



[INFO] MCP tool "multiply_tool"/"multiply" received input: {'a': 6, 'b': 9}
[INFO] MCP tool "multiply_tool"/"multiply" received result (size: 2)
[INFO] MCP server "multiply_tool": session closed
[INFO] MCP server "add_tool": session closed


Query: Compute 6 × 9.
Answer: The result of multiplying 6 and 9 is 54.

